## 加载必要的包

In [1]:
import numpy as np
import netCDF4 as nc

## 读取nc文件



In [2]:
nc_data = nc.Dataset("lftx.sfc.2021.nc")

## 查看nc文件的维度

In [3]:
for temp_dim in nc_data.dimensions.values():
    print(f"dimension name: {temp_dim.name}, dimension size: {temp_dim.size}")

dimension name: lat, dimension size: 73
dimension name: lon, dimension size: 144
dimension name: time, dimension size: 306
dimension name: nbnds, dimension size: 2


## 查看nc文件的变量


In [4]:
for temp_value in nc_data.variables.values():
    print("*" * 70, '\n')
    # print(f"variable name: {temp_value.name}, variable shape: {temp_value.shape}")
    print(temp_value)

********************************************************************** 

<class 'netCDF4._netCDF4.Variable'>
float32 lat(lat)
    units: degrees_north
    actual_range: [ 90. -90.]
    long_name: Latitude
    standard_name: latitude
    axis: Y
unlimited dimensions: 
current shape = (73,)
filling on, default _FillValue of 9.969209968386869e+36 used
********************************************************************** 

<class 'netCDF4._netCDF4.Variable'>
float32 lon(lon)
    units: degrees_east
    long_name: Longitude
    actual_range: [  0.  357.5]
    standard_name: longitude
    axis: X
unlimited dimensions: 
current shape = (144,)
filling on, default _FillValue of 9.969209968386869e+36 used
********************************************************************** 

<class 'netCDF4._netCDF4.Variable'>
float64 time(time)
    long_name: Time
    delta_t: 0000-00-01 00:00:00
    standard_name: time
    axis: T
    units: hours since 1800-01-01 00:00:0.0
    avg_period: 0000-00-01 00:00:

<ipython-input-4-0cd40c49039b>:4: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  print(temp_value)


## 提取变量

In [5]:
process_data = np.array(nc_data.variables['lftx'])  # (time, lat, lon)
process_data.shape

<ipython-input-5-e18014418500>:1: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  process_data = np.array(nc_data.variables['lftx'])  # (time, lat, lon)


(306, 73, 144)

In [6]:
(time, lat, lon) = process_data.shape

## 数据处理
我们求各个位置的平均数

In [7]:
result_data = np.zeros(shape=(lat, lon))

from tqdm import tqdm

for temp_lat in tqdm(range(lat)):
    for temp_lon in range(lon):
        result_data[temp_lat, temp_lon] = process_data[:, temp_lat, temp_lon].sum()

print(f"result data shape: {result_data.shape}")


100%|██████████| 73/73 [00:00<00:00, 1378.95it/s]

result data shape: (73, 144)


In [8]:
lat_data = np.array(nc_data.variables['lat'])  # degrees_north

lon_data = np.array(nc_data.variables['lon'])  # degrees_east

<ipython-input-8-63bd4a3b8296>:1: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  lat_data = np.array(nc_data.variables['lat'])  # degrees_north
<ipython-input-8-63bd4a3b8296>:3: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  lon_data = np.array(nc_data.variables['lon'])  # degrees_east


In [9]:
Lon_data, Lat_data = np.meshgrid(lon_data, lat_data)

Lon_data.shape

(73, 144)

In [10]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

ax.plot_surface(Lon_data, Lat_data, result_data, cmap=plt.cm.YlGnBu_r)

ax.set_xlabel(r'Longitude (degrees_east)')
ax.set_ylabel(r'Latitude (degrees_north)')
ax.set_zlabel(r'sum (degK)')

plt.show()

Using matplotlib backend: MacOSX
